#### PCA, K-means and plotting
In this notebook, the kmeans clustering worked! Here, it is implemented in the same form as the "de.pca_hierarchical_plot" function. This way, both clustering methods can be compared visually.

In [ ]:
import numpy as np
import data_cleanup as dc
import data_exploration as de
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA


aa = pd.read_csv('../DMS_data/aminoacids.csv')

gia_null_eto: pd.DataFrame = pd.read_csv('../DMS_data/P53_HUMAN_Giacomelli_NULL_Etoposide_2018.csv')
norm_frame = dc.aufteilung_mut_pos(dc.norm(gia_null_eto))

In [ ]:
dist_chem = de.aa_distance_matrix(aa)
dist_p53 = de.dms_distance_matrix_wt(norm_frame)

In [ ]:
def determine_clusters_kmeans(data, max_clusters):
    silhouette_scores = []

    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        silhouette_avg = silhouette_score(data, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Plotting the Silhouette scores
    #plt.figure(figsize=(10, 6))
    #plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
    #plt.xlabel('Number of Clusters')
    #plt.ylabel('Average Silhouette Score')
    #plt.title('Silhouette Method: Determining Optimal Number of Clusters')
    #plt.show()

    # Find the optimal number of clusters using the maximum Silhouette score
    optimal_clusters = np.argmax(silhouette_scores) + 1  # Adding 1 to account for zero-based indexing

    return optimal_clusters


# Example usage
data = dc.rmv_na(dc.df_transform(dc.norm(gia_null_eto)))
determine_clusters_kmeans(data, max_clusters=10)


In [ ]:
def pca_kmeans_plot (dist_matrix: pd.DataFrame, optimal_num_cluster: int, title: str):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(dist_matrix.values)

    kmeans = KMeans(n_clusters = optimal_num_cluster, random_state=42)
    clusters = kmeans.fit_predict(pca_result)

    df = pd.DataFrame(data = pca_result, columns=['PC1', 'PC2'])
    df['Cluster'] = clusters
    df['Label'] = dist_matrix.index

    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=df, x='PC1', y='PC2', hue='Cluster', palette='Set1').set(
    title=f"PCA with K-means Clustering of {title} ")

    for i, point in df.iterrows():
        plt.annotate(point['Label'], (point['PC1'], point['PC2']), textcoords="offset points", xytext=(0, -10), ha='center')

    plt.legend(title='Cluster', loc='lower right')
    return plt.show()

pca_kmeans_plot(dist_p53,4 , title = "p53 distance matrix")

 just to possibly "overwrite" Fridos push on the Documentation